<a href="https://colab.research.google.com/github/euyju/AI-2025/blob/main/202011086%EC%9D%B4%EC%9D%98%EC%A3%BC_%EB%B6%84%EB%A5%98%EC%8B%A4%EC%8A%B5%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.분류


In [4]:
# 분류 문제: MNIST 손글씨 이미지를 0~9 숫자 중 하나로 분류
# 목표: 28x28 크기의 이미지를 784 벡터로 변환하여 다중 클래스 분류 수행

#2. 데이터셋

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

# (28x28) 70K개
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.to_numpy()
Y = mnist.target.astype(int)

#정규화
X = X / 255.0

#트레이닝 셋, 테스트 셋 구분
X_train, X_test, Y_train, Y_test = train_test_split(
  X, Y, test_size=0.3, random_state=42)

Y_train = Y_train.to_numpy()
Y_test = Y_test.to_numpy()

print("전체 데이터 개수:", X.shape[0])
print("훈련용 데이터 개수:", X_train.shape[0])
print("테스트용 데이터 개수:", X_test.shape[0])

전체 데이터 개수: 70000
훈련용 데이터 개수: 49000
테스트용 데이터 개수: 21000


# 3. 모델설정

In [18]:
class LogisticRegressionOVR:
    def __init__(self, lr=0.1, epochs=1000):
        self.lr = lr
        self.epochs = epochs
        self.weights = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def compute_loss(self, y, y_hat):
        m = len(y)
        return -1/m * np.sum(y * np.log(y_hat + 1e-8) +
                             (1 - y) * np.log(1 - y_hat + 1e-8))

    def fit(self, X, y):
        m, n = X.shape
        X = np.hstack([np.ones((m, 1)), X])
        self.weights = np.zeros((10, n + 1))

        for i in range(self.epochs):
            for c in range(10):
                y_c = (y == c).astype(int).reshape(-1, 1)
                z = np.dot(X, self.weights[c].reshape(-1, 1))
                y_hat = self.sigmoid(z)
                gradient = (1/m) * np.dot(X.T, (y_hat - y_c))
                self.weights[c] -= self.lr * gradient.flatten()

    def predict(self, X):
        m = X.shape[0]
        X = np.hstack([np.ones((m, 1)), X])
        logits = np.dot(X, self.weights.T)
        probabilities = self.sigmoid(logits)
        return np.argmax(probabilities, axis=1)

# 4. 모델 훈련

In [19]:
model = LogisticRegressionOVR(lr=0.1, epochs=1000)
losses = model.fit(X_train, Y_train)

# 5. 모델 검증

In [20]:
Y_pred = model.predict(X_test)

# Confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:\n", cm)

# F1 Score 출력
f1 = f1_score(Y_test, Y_pred, average='macro')
print("F1 Score (macro):", f1)

Confusion Matrix:
 [[1986    1    4    1    6   10   23    2   23    2]
 [   0 2292    6   14    2   12    1    8   27    2]
 [  16   33 1829   32   35    4   53   39   73   19]
 [  11   18   59 1897    2   59   26   27   49   28]
 [   3   11   16    4 1757    3   13    5   21  103]
 [  43   23   12  115   41 1498   37   12   97   37]
 [  10    8   21    4   25   29 1979    2   10    0]
 [  17   26   40   10   25    4    0 2054    7   65]
 [  16   64   19   67   16   56   23   15 1671   45]
 [  20   16   19   32   75   16    0   78   26 1808]]
F1 Score (macro): 0.8918271435552135
